In [5]:
import pandas as pd

# predictions = pd.read_csv('fn1.7-test-gpt-4o-mini-tempurature-0-xml-descriptions.csv')
# predictions = pd.read_csv('fn1.7-test-gpt-4o-mini-tempurature-1-xml-descriptions-p1.csv')
predictions = pd.read_csv('fn1.7-test-gpt-4o-mini-tempurature-1-xml-descriptions-p2.csv')



In [6]:
import re
def get_tags(text: str) -> dict:
    # example: 'Your gift to Goodwill is important because <Cognizer>people with physical and Mental disabilities</Cognizer> sometimes need an extra hand to <Entity>know</Entity> the pride that comes with work .'
    # output: {'Cognizer': 'people with physical and Mental disabilities', 'Entity': 'know'}
    tags = {}

    # get all text between tags
    matches = re.findall(r'<(.*?)>(.*?)</(.*?)>', text)
    for match in matches:
        tag = match[0]
        content = match[1]
        tags[tag] = content

    return tags


In [8]:
tp = 0
near_miss = 0
fp = 0
fn = 0
tn = 0 # not really used because tags only have positive values, technically all other FEs are TN, but we don't really care about that (maybe we should?)

for label, pred, fes in predictions[['output', 'prediction', 'frame_elements']].values:

    # Get each predicted FE span from the prediction
    pred_tags = get_tags(pred)
    real_tags = eval(fes)

    # Check each predicted FE span
    for tag, content in pred_tags.items():
        if tag.capitalize() in real_tags:
            if content == real_tags[tag.capitalize()]:
                tp += 1
            else:
                fp += 1
                near_miss += 1
        else:
            fp += 1
    
    # Check each real FE span
    for tag, content in real_tags.items():
        if tag not in pred_tags:
            fn += 1
        elif content != pred_tags[tag]:
            fn += 1

# print(f'Perfect: {perfect} out of {len(predictions)}')
print(f'TP: {tp}')
print(f'Missed: {near_miss}')
print(f'FP: {fp}')
print(f'FN: {fn}')

print(f'Precision: {tp / (tp + fp)}')
print(f'Precision (ignoring near misses): {tp / (tp + fp - near_miss)}')
print(f'Recall: {tp / (tp + fn)}')
print(f'F1: {2 * tp / (2 * tp + fp + fn)}')

print(f'Accuracy: {tp / (tp + fp + fn)}')


TP: 3955
Missed: 3621
FP: 8819
FN: 6994
Precision: 0.30961327696884294
Precision (ignoring near misses): 0.43209876543209874
Recall: 0.36122020275824274
F1: 0.33343169076423723
Accuracy: 0.20007082152974504
